In [1]:
using Plots
using LinearAlgebra
plotly()

┌ Warning: Failed to load integration with PlotlyBase & PlotlyKaleido.
│   exception =
│    ArgumentError: Package PlotlyKaleido not found in current path.
│    - Run `import Pkg; Pkg.add("PlotlyKaleido")` to install the PlotlyKaleido package.
│    Stacktrace:
│      [1] top-level scope
│        @ ~/.julia/packages/Plots/kLeqV/src/backends.jl:569
│      [2] eval
│        @ ./boot.jl:385 [inlined]
│      [3] _initialize_backend(pkg::Plots.PlotlyBackend)
│        @ Plots ~/.julia/packages/Plots/kLeqV/src/backends.jl:567
│      [4] backend
│        @ ~/.julia/packages/Plots/kLeqV/src/backends.jl:245 [inlined]
│      [5] plotly()
│        @ Plots ~/.julia/packages/Plots/kLeqV/src/backends.jl:86
│      [6] top-level scope
│        @ In[1]:3
│      [7] eval
│        @ ./boot.jl:385 [inlined]
│      [8] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
│        @ Base ./loading.jl:2076
│      [9] softscope_include_string(m::Module, code::String, file

Plots.PlotlyBackend()

In [2]:
function approximate_gradient(f, x, h=1e-8)
    n = length(x)
    grad = zeros(n)
    
    for i in 1:n
        x_plus_h = copy(x)
        x_plus_h[i] += h
        
        x_minus_h = copy(x)
        x_minus_h[i] -= h
        
        grad[i] = (f(x_plus_h) - f(x_minus_h)) / (2*h)
    end
    
    return grad
end

function golden_section_search(f, a, b, tol=1e-6, max_iter=100)
    golden_ratio = (sqrt(5) + 1) / 2
    c = b - (b - a) / golden_ratio
    d = a + (b - a) / golden_ratio
    
    fc = f(c)
    fd = f(d)
    
    iter = 0
    while abs(b - a) > tol && iter < max_iter
        iter += 1
        
        if fc < fd
            b = d
            d = c
            fd = fc
            c = b - (b - a) / golden_ratio
            fc = f(c)
        else
            a = c
            c = d
            fc = fd
            d = a + (b - a) / golden_ratio
            fd = f(d)
        end
    end
    
    return (a + b) / 2
end

function swann_method(f, x0, h=0.1)
    first = x0
    second = x0 + h
    if f(second) > f(first)
        h = -h
        first, second = second, second + h
    end
    last = second + h
    
    while f(last) < f(second)
        h *= 2
        first, second, last = second, last, last + h
    end
    if second > last
        first, second, last = last, second, first
    end

    return first, last
end

swann_method (generic function with 2 methods)

In [3]:
function bfgs(f, x0; tol=1e-4, max_iter=10000)
    x = x0
    n = length(x)
    H = I(n)
    println(H)
    trajectory = [x]

    for _ in 1:max_iter
        g = approximate_gradient(f, x)
        if norm(g) < tol
            break
        end
        
        p = -H * g
        ff(alpha) = f(x + alpha * p)
        a, b = swann_method(ff, 0.0)
        alpha = golden_section_search(ff, a, b)
        
        x_new = x + alpha * p
        delta_x = x_new - x
        delta_g = approximate_gradient(f, x_new) - g
        
        if dot(delta_x, delta_g) > 0
            rho = 1.0 / dot(delta_x, delta_g)
            I_n = I(n)
            H = (I_n - rho * delta_x * delta_g') * H * (I_n - rho * delta_g * delta_x') + rho * delta_x * delta_x'
            println(H)
        end
        
        x = x_new
        push!(trajectory, x)
    end
    
    return x, trajectory
end

bfgs (generic function with 1 method)

In [4]:
function dfp(f, x0; tol=1e-4, max_iter=1000)
    x = x0
    n = length(x)
    H = I(n)  # Начальная матрица H - единичная
    println(H)
    trajectory = [x]  # Список для хранения траектории

    for iter in 1:max_iter
        g = approximate_gradient(f,x)
        d = -H * g  # Направление спуска

        ff(alpha) = f(x + alpha * d)
        a, b = swann_method(ff, 0.0)
        alpha = golden_section_search(ff, a, b)
        
        x_new = x + alpha * d
        g_new = approximate_gradient(f,x_new)
        
        s = x_new - x
        y = g_new - g

        # Обновление матрицы H
        ys = y' * s
        H = H + (s * s') / ys - (H * (y * y') * H) / (y' * H * y)
        println(H)

        # Добавление новой точки в траекторию
        push!(trajectory, x_new)

        # Проверка на сходимость
        if norm(g_new) < tol
            return x_new, trajectory  # Возвращаем минимум и траекторию
        end
        
        x = x_new
    end
    
    return x, trajectory  # Возвращаем минимум и траекторию
end

dfp (generic function with 1 method)

In [5]:
function plot_optimization_paths(f, x_range, y_range, paths_with_names, title="Сравнение методов оптимизации", global_min=nothing)
    z = [f([x, y]) for y in y_range, x in x_range]
    
    clamp_level = maximum(filter(isfinite, z)) / 2 
    z_clamped = [min(val, clamp_level) for val in z]
    
    p = contour(x_range, y_range, z_clamped,
                fill=false,
                levels=20,
                color=:thermal,
                xlabel="x",
                ylabel="y",
                title=title,
                size=(800, 600))
    
    colors = [:red, :green, :blue, :purple, :orange]
    for (i, (name, path)) in enumerate(paths_with_names)
        x_coords = [point[1] for point in path]
        y_coords = [point[2] for point in path]
        
        plot!(p, x_coords, y_coords, 
              label=name, 
              line=(colors[i], 2),
              marker=(:circle, 2, 0.5))
        
        annotate!(p, x_coords[1], y_coords[1], text("Старт", :left, 8, :white))
        annotate!(p, x_coords[end], y_coords[end], text("Финиш", :right, 8, :white))
    end
    
    if global_min !== nothing
        scatter!(p, [global_min[1]], [global_min[2]], 
                label="Глобальный минимум", 
                color=:white, 
                markersize=5,
                markerstrokewidth=1,
                markerstrokecolor=:black)
    end
    
    return p
end

plot_optimization_paths (generic function with 3 methods)

In [6]:
# 1. Функция Розенброка
function rosenbrock(x)
    return (1.0 - x[1])^2 + 100.0*(x[2] - x[1]^2)^2
end

# 2. Функция Растригина
function rastrigin(x)
    return 20 + x[1]^2 - 10*cos(2*pi*x[1]) + x[2]^2 - 10*cos(2*pi*x[2])
end

# 3. Функция Швефеля
function schwefel(x)
    return 418.9829*2 - (x[1]*sin(sqrt(abs(x[1]))) + x[2]*sin(sqrt(abs(x[2]))))
end

function my(x)
    return (x[1] - 4*x[2])^2 + (x[2] + 5)^2
end

a = 7.0
b = 5.0
function quadratic(x)
    return a * x[1]^2 + b * x[2]^2
end


function run_optimization(f, x0, title, x_range, y_range, global_min=nothing)
    println("\n======= $(title) =======")
    
    result_bfgs, path_bfgs = bfgs(f, x0)
    
    println("Метод BFGS (Бройдена, Флэтчера, Гольдфарба, Шанно) (результат): ", result_bfgs)
    println("Значение функции: ", f(result_bfgs))
    println("Количество итераций: ", length(path_bfgs))
    println()

    result_dfp, path_dfp = dfp(f, x0)
    println("Метод DFP (результат): ", result_dfp)
    println("Значение функции: ", f(result_dfp))
    println("Количество итераций: ", length(path_dfp))
    println()
    
    paths = [
        ("BFGS", path_bfgs),
        ("DFP",path_dfp),
    ]
    
    p = plot_optimization_paths(f, x_range, y_range, paths, title, global_min)
    
    display(p)
    
    return nothing
end

run_optimization (generic function with 2 methods)

In [7]:
# Для функции Розенброка
x0_rosenbrock = [-1.2, 2.0]
x_range_rosenbrock = -3.0:0.1:3.0
y_range_rosenbrock = -1.0:0.1:4.0
global_min_rosenbrock = [1.0, 1.0]
run_optimization(rosenbrock, x0_rosenbrock, "Функция Розенброка", x_range_rosenbrock, y_range_rosenbrock, global_min_rosenbrock)


======= Функция Розенброка =======
Bool[1 0; 0 1]
[0.14531077539600373 0.5249552975699511; 0.5249552975699512 1.9014688625570382]
[0.13447211905600925 0.47236695394001843; 0.4723669539400158 1.6627907097503154]
[0.08841181777104266 0.27701186584004145; 0.27701186584004134 0.8777875313851774]
[0.12718914867684908 0.3846589122780521; 0.38465891227805143 1.1682582491048674]
[0.1603864555030528 0.4746175949694132; 0.4746175949694042 1.4075121882599555]
[0.12096952519389567 0.326155855107225; 0.3261558551072239 0.8850208750426758]
[0.2504728858379345 0.6185050480253659; 0.6185050480253648 1.544987300234274]
[0.07433171795104242 0.15335154982271548; 0.15335154982271637 0.32093233762777146]
[0.501701225798488 1.0011955091158986; 1.0011955091158986 2.0029335460019246]
[0.5082547014889816 1.0163427370482132; 1.0163427370482145 2.0373545748973863]
Метод BFGS (Бройдена, Флэтчера, Гольдфарба, Шанно) (результат): [0.9999940286100322, 0.9999881549597779]
Значение функции: 3.661210640510455e-11
Коли

In [8]:
# Для функции Швефеля
x0_schwefel = [100.5, 100.5]
x_range_schwefel = -500.0:10.0:500.0
y_range_schwefel = -500.0:10.0:500.0
global_min_schwefel = [420.9687, 420.9687]
run_optimization(schwefel, x0_schwefel, "Функция Швефеля", x_range_schwefel, y_range_schwefel, global_min_schwefel)


======= Функция Швефеля =======
Bool[1 0; 0 1]
[4.2173310882728545 3.2173310882728545; 3.2173310882728545 4.2173310882728545]
Метод BFGS (Бройдена, Флэтчера, Гольдфарба, Шанно) (результат): [65.5478640218449, 65.5478640218449]
Значение функции: 710.6958360968913
Количество итераций: 2

Bool[1 0; 0 1]
[4.2173310882728545 3.2173310882728545; 3.2173310882728545 4.2173310882728545]
Метод DFP (результат): [65.5478640218449, 65.5478640218449]
Значение функции: 710.6958360968913
Количество итераций: 2



In [9]:
# Для функции Швефеля
x0_my = [10.0, 10.0]
x_range_my = -25.0:1.0:20.0
y_range_my = -25.0:1.0:20.0
global_min_my = [-20, -5]
run_optimization(my, x0_my, "Функция (x[1] - 4*x[2])^2 + (x[2] + 5)^2", x_range_my, y_range_my, global_min_my)


======= Функция (x[1] - 4*x[2])^2 + (x[2] + 5)^2 =======
Bool[1 0; 0 1]
[0.9487062655793242 0.21770706060358208; 0.21770706060358205 0.07933458155635822]
[8.499999400003151 1.9999998589115646; 1.9999998589115642 0.4999999638582372]
[8.499999324525254 2.000000018715135; 2.000000018715135 0.5000000465127683]
Метод BFGS (Бройдена, Флэтчера, Гольдфарба, Шанно) (результат): [-20.0000000016891, -5.000000000427331]
Значение функции: 1.8302090920497276e-19
Количество итераций: 4

Bool[1 0; 0 1]
[0.9485433447547602 0.21766860724281165; 0.21766860724281165 0.07932550560821894]
[8.499999399730664 1.9999998601208888; 1.9999998601208888 0.4999999584932686]
[8.499999323204275 2.0000000184067286; 2.0000000184067246 0.5000000464430658]
Метод DFP (результат): [-20.000000001701416, -5.000000000375315]
Значение функции: 1.8092382568485974e-19
Количество итераций: 4



In [10]:
x0_quadratic = [-2.0, -2.0]
x_range_quadratic = -3.0:0.1:3.0
y_range_quadratic = -3.0:0.1:3.0
global_min_quadratic = [0.0, 0.0]
run_optimization(quadratic, x0_quadratic, "Квадратичная функция", x_range_quadratic, y_range_quadratic, global_min_quadratic)


======= Квадратичная функция =======
Bool[1 0; 0 1]
[0.26351452028010136 -0.3764884341724138; -0.37648843417241373 0.8379172840767705]
[0.0714285719086413 3.441454829444801e-10; 3.441454690666923e-10 0.10000000003651255]
Метод BFGS (Бройдена, Флэтчера, Гольдфарба, Шанно) (результат): [-2.185549069583237e-6, -2.5868677738793977e-7]
Значение функции: 3.377096739286986e-11
Количество итераций: 3

Bool[1 0; 0 1]
[0.2588937386247364 -0.3674317026860728; -0.3674317026860728 0.820166091457563]
[0.07142857190853286 3.440679963162552e-10; 3.440680518274064e-10 0.10000000003645615]
Метод DFP (результат): [-1.2107133310623741e-6, -2.159300712678558e-6]
Значение функции: 3.357368522895569e-11
Количество итераций: 3

